In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from PIL import Image
import pytesseract
from googletrans import Translator
from langdetect import detect
import re

In [2]:
def remove_special_characters(text):
    # Define the pattern for special characters
    pattern = r'[^a-zA-Z0-9\s]'
    
    # Remove special characters using regex
    text = re.sub(pattern, '', text)
    
    return text

In [3]:
def get_first_word(text):
    # Split the text into words
    words = text.split()
    
    # Retrieve the first word
    if len(words) > 0:
        first_word = words[0]
        return first_word
    
    # Return None if the text is empty or contains no words
    return None

In [4]:
input_string= 'C:/Users/user/Desktop/jupyter notebook/napa_pac.png'
dataset_csv = 'C:/Users/user/Desktop/jupyter notebook/medicine.csv'
dataset_image = 'C:/Users/user/Desktop/jupyter notebook/image_dataset'

In [5]:
image = Image.open(input_string)

In [6]:
textOfImage = pytesseract.image_to_string(image, lang='eng') # for all latin languages
# textOfImage = pytesseract.image_to_string(image, lang='chi_sim') #for non-latin chinease languages 
clean_text = remove_special_characters(textOfImage)
print(get_first_word(clean_text))
text_after_word= get_first_word(clean_text)
# english_string = convert_to_english(textOfImage)
print(text_after_word)

Napa
Napa


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import pyttsx3

# Read the dataset
dataset_path = dataset_csv
data = pd.read_csv(dataset_path)

# Drop rows with missing values
data = data.dropna()

# Separate X and y
X = data['brand name']
y = data[['type', 'slug', 'dosage form', 'generic', 'strength']]

# Initialize label encoders and encode y
label_encoders = {}
y_encoded = {}

for col in y.columns:
    label_encoders[col] = LabelEncoder()
    y_encoded[col] = label_encoders[col].fit_transform(y[col])

# Convert y_encoded to a DataFrame and reset indices to align X and y
y_encoded = pd.DataFrame(y_encoded)
X = X.reset_index(drop=True)
y_encoded = y_encoded.reset_index(drop=True)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Tokenize and pad the text sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

max_sequence_length = max(len(seq) for seq in X_train_sequences) + 1
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

models = {}
for col in y.columns:
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(label_encoders[col].classes_.shape[0], activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train_padded, y_train[col], validation_data=(X_test_padded, y_test[col]), epochs=10, batch_size=32)
    models[col] = model

medicine_name = text_after_word
medicine_name_sequence = tokenizer.texts_to_sequences([medicine_name])
medicine_name_padded = pad_sequences(medicine_name_sequence, maxlen=max_sequence_length)

predicted_purposes_decoded = {}
overall_confidence = 0.0
num_columns = 0

for col, model in models.items():
    predicted_purposes = model.predict(medicine_name_padded)
    predicted_purposes_decoded[col] = label_encoders[col].inverse_transform(predicted_purposes.argmax(axis=1))
    confidence = predicted_purposes.max(axis=1)[0] * 100
    overall_confidence += confidence
    num_columns += 1

overall_confidence_percentage = overall_confidence / num_columns

engine = pyttsx3.init()
engine.setProperty('rate', 150)

overall_text = f"The overall prediction is {overall_confidence_percentage:.2f}%."

# Use the engine to speak the overall prediction
engine.say(overall_text)
engine.runAndWait()

print(overall_text)

Epoch 1/10
337/337 [==============================] - 6s 14ms/step - loss: 0.1005 - accuracy: 0.9808 - val_loss: 0.0936 - val_accuracy: 0.9800
Epoch 2/10
337/337 [==============================] - 5s 14ms/step - loss: 0.0221 - accuracy: 0.9933 - val_loss: 0.8101 - val_accuracy: 0.6614
Epoch 3/10
337/337 [==============================] - 5s 13ms/step - loss: 0.0033 - accuracy: 0.9994 - val_loss: 1.2501 - val_accuracy: 0.6543
Epoch 4/10
337/337 [==============================] - 4s 13ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 1.0473 - val_accuracy: 0.6621
Epoch 5/10
337/337 [==============================] - 4s 13ms/step - loss: 4.3002e-04 - accuracy: 0.9999 - val_loss: 1.4331 - val_accuracy: 0.6621
Epoch 6/10
337/337 [==============================] - 4s 13ms/step - loss: 3.3651e-05 - accuracy: 1.0000 - val_loss: 1.6242 - val_accuracy: 0.6621
Epoch 7/10
337/337 [==============================] - 4s 12ms/step - loss: 1.6728e-05 - accuracy: 1.0000 - val_loss: 1.7807 - val_accu